In [ ]:
%pip install transformers 
%pip install sklearn
%pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments,DataCollatorWithPadding
from datasets import load_dataset, load_metric,Dataset
import torch
import numpy as np
from utils import AssessData
from datasets import load_dataset
from utils import PrepareCorpus,AssessData
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset
import json
from re import template
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
parentdir = "/content/drive/MyDrive/ML/Datasets/20News"

train =parentdir+"/20news-bydate"+"/20news-bydate-train"
test = parentdir+"/20news-bydate"+ "/20news-bydate-test"

In [ ]:
id2labels = {'talk.politics.mideast': 0, 'rec.autos': 1, 'comp.sys.mac.hardware': 2, 'alt.atheism': 3, 'rec.sport.baseball': 4, 'comp.os.ms-windows.misc': 5, 'rec.sport.hockey': 6, 'sci.crypt': 7, 'sci.med': 8, 'talk.politics.misc': 9, 'rec.motorcycles': 10, 'comp.windows.x': 11, 'comp.graphics': 12, 'comp.sys.ibm.pc.hardware': 13, 'sci.electronics': 14, 'talk.politics.guns': 15, 'sci.space': 16, 'soc.religion.christian': 17, 'misc.forsale': 18, 'talk.religion.misc': 19}
id2classes = {'talk.politics.mideast': 0, 'rec.autos': 4, 'comp.sys.mac.hardware': 2, 'alt.atheism': 5, 'rec.sport.baseball': 1, 'comp.os.ms-windows.misc': 2, 'rec.sport.hockey': 1, 'sci.crypt': 3, 'sci.med': 3, 'talk.politics.misc': 0, 'rec.motorcycles': 4, 'comp.windows.x': 2, 'comp.graphics': 2, 'comp.sys.ibm.pc.hardware': 2, 'sci.electronics': 3, 'talk.politics.guns': 0, 'sci.space': 3, 'soc.religion.christian': 5, 'misc.forsale': 6, 'talk.religion.misc': 5}

max_input_length = 300
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

number = range(50,150,50)
_per_segment = range(200,400,100)

overlap = {"side":"both", "number": 50}

f1_score = load_metric("f1"); precision = load_metric("precision"); recall = load_metric("recall")

def tokenize(batch):
    return bert_tokenizer(batch['text'], truncation=True, max_length=max_input_length,padding = True)

def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return {"f1" :f1_score.compute(predictions = predictions, references = label, average = 'weighted'),
            "precision" : precision.compute(predictions = predictions, references = label, average = 'weighted'),
            "recall": recall.compute(predictions = predictions, references = label, average = 'weighted')}

def hp_space(trial):
  return {"per_device_train_batch_size": trial.suggest_discrete_uniform("per_device_train_batch_size", 8,32,8)
        ,"learning_rate": trial.suggest_float("learning_rate", 0.00001,0.00005, log = True)
        ,"num_train_epochs": trial.suggest_int("num_train_epochs",1,10)}

def bert_init():
  return BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 7)

def id_2_labels(x,adict):
  """converts labels/classes into a number using a dictionary"""
  return adict[x]

def _indexing(alist):
  id = 0; indexed = {}
  for x in alist:
    for y in x:
      id +=1
      indexed[id] = y
  return indexed
  

In [ ]:
news_group_train = PrepareCorpus(train)
news_group_test = PrepareCorpus(test)

In [ ]:
news_group_test = news_group_test._prep()       #Keys are labels, values are texts
news_group_train = news_group_train._prep()

SUBCATEGORY MAPPING

In [ ]:
#select only politics
id_politics = {'talk.politics.misc': 0, 'talk.politics.mideast': 1, 'talk.politics.guns': 2}
id_religion = {'alt.atheism': 0, 'soc.religion.christian' : 1, 'talk.religion.misc': 2}
id_sport = {'rec.sport.baseball': 0, 'rec.sport.hockey': 1}
id_autos = {'rec.motorcycles': 0, 'rec.autos': 1,}
id_sci = {'sci.crypt': 0, 'sci.med': 1, 'sci.space': 2}
id_comp = {'comp.sys.mac.hardware': 0,  'comp.os.ms-windows.misc': 1, 'comp.windows.x': 2, 'comp.graphics': 3, 'comp.sys.ibm.pc.hardware': 4, 'sci.electronics': 5}
id_sale = {'misc.forsale': 0}

In [ ]:
#Selecting id_sci

temp_train = []; temp_test = []
temp_news_group_train = {}; temp_news_group_test = {}

for item in id_sci.keys():#change id as required 
  temp_news_group_train[item] = news_group_train.get(item)
  temp_news_group_test[item] = news_group_test.get(item)

temp_train = _indexing(temp_news_group_train.values())
temp_test =  _indexing(temp_news_group_test.values())

temp_train_corpus = AssessData(temp_train, temp_news_group_train)
temp_test_corpus = AssessData(temp_test, temp_news_group_test)

In [ ]:
type(temp_train_corpus)

utils.AssessData

In [ ]:
temp_train_index_to_label = temp_train_corpus._index_to_label() 
temp_test_index_to_label = temp_test_corpus._index_to_label()

In [ ]:
set(temp_train_index_to_label.values())

{'sci.crypt', 'sci.med', 'sci.space'}

In [ ]:
temp_train_index_to_label = {i:id_2_labels(x, id_sci) for i,x in temp_train_index_to_label.items()} #change id as required
temp_test_index_to_label = {i:id_2_labels(x, id_sci) for i,x in temp_test_index_to_label.items()}

In [ ]:
temp_train_set = temp_train_corpus._chunk(200, overlap=overlap)
temp_train_index, temp_train_text = zip(*temp_train_set)

temp_test_set = temp_test_corpus._chunk(200, overlap=overlap)
test_index, test_text = zip(*temp_test_set)

In [ ]:
train_index = {a:x-1 for a,x in enumerate(temp_train_index)}
train_text = {a:x for a,x in enumerate(temp_train_text)}

In [ ]:
test_index = {a:x-1 for a,x in enumerate(test_index)}
test_text = {a:x for a,x in enumerate(test_text)}

In [ ]:
train_chunked_label = {i:id_2_labels(x,temp_train_index_to_label) for i,x in train_index.items()}
test_chunked_label = {i:id_2_labels(x,temp_test_index_to_label) for i,x in test_index.items()}

In [ ]:
train_chunked_label

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 0,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 0,
 117: 0,
 118: 0,
 119: 0,
 120: 0,
 121: 0,
 122: 0,
 12

In [ ]:
sci_train_set = {"train" : {"text": train_text.values(), 'doc_id': train_index.values(), 'labels': train_chunked_label.values()}}
sci_test_set =  {"test" : {"text": test_text.values(), 'doc_id': test_index.values(), 'labels': test_chunked_label.values()}}

In [ ]:
import pandas as pd

In [ ]:
a = pd.DataFrame(sci_train_set['train'])
b = pd.DataFrame(sci_test_set['test'])

In [ ]:
a

,text,doc_id,labels
0,From: WHMurray@DOCKMASTER.NCSC.MIL\n Subject: ...,0,0
1,backdoor in\n >RSAREF and IF PKP is supported ...,0,0
2,lves against ANY\n >implementation that doesn'...,0,0
3,limitation in the concept of RSAREF in which\n...,0,0
4,. \n \n It has a legitimate (literally) right...,0,0
...,...,...,...
22529,.. (okay they use modern methods toa point).\n...,1781,2
22530,re\n many of these people do not sustaint heir...,1781,2
22531,ated inthe 1830's-1870's when the west was\n ...,1781,2
22532,the same industry forever. Anyone who claims ...,1781,2


In [ ]:
a['text'] = a['text'].astype(str)
a['doc_id'] = a['doc_id'].astype(int)
a['labels'] = a['labels'].astype(int)

b['text'] = a['text'].astype(str)
b['doc_id'] = a['doc_id'].astype(int)
b['labels'] = a['labels'].astype(int)

In [ ]:
sci_train_set = Dataset.from_pandas(a)
sci_test_set = Dataset.from_pandas(b)

In [ ]:
sci_train_set = sci_train_set.map(tokenize)
sci_test_set = sci_test_set.map(tokenize)

sci_train_set = sci_train_set.shuffle(seed = 24)
sci_test_set = sci_test_set.shuffle(seed = 24)

  0%|          | 0/22534 [00:00<?, ?ex/s]

  0%|          | 0/12846 [00:00<?, ?ex/s]

In [ ]:
sci_test_set

Dataset({
    features: ['text', 'doc_id', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12846
})

In [ ]:
fold = KFold(n_splits = 5)
this_set = fold.split(sci_train_set['text']) #change

frac_fold_train = []; frac_fold_eval = []

for i,j in this_set:
  frac_fold_train.append(sci_train_set.select(i))
  frac_fold_eval.append(sci_train_set.select(j))

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer, max_length= 300)

bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = len(set(a['labels'])))

#default training arguments 
training_args = TrainingArguments(output_dir="./frac_results", learning_rate=2e-5, per_device_train_batch_size=16, per_device_eval_batch_size=16, num_train_epochs=5,weight_decay=0.01)
results = []

#fine-tune
trainer = Trainer(model = bert_model, 
                args = training_args,
                train_dataset= frac_fold_train[0],
                eval_dataset= frac_fold_eval[0],
                tokenizer= bert_tokenizer,
                data_collator=data_collator,
                compute_metrics= compute_metrics)

trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss
500,0.435600
1000,0.299600
1500,0.214100
2000,0.191400
2500,0.153500
3000,0.127500
3500,0.128600
4000,0.111400
4500,0.097400
5000,0.093100


Saving model checkpoint to ./frac_results/checkpoint-500
Configuration saved in ./frac_results/checkpoint-500/config.json
Model weights saved in ./frac_results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./frac_results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./frac_results/checkpoint-500/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2340: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
Saving model checkpoint to ./frac_results/checkpoint-1000
Configuration saved in ./frac_results/checkpoint-1000/config.json
Model weights saved in ./frac_results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./frac_results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./frac_result

TrainOutput(global_step=5635, training_loss=0.17489797343378483, metrics={'train_runtime': 1096.1593, 'train_samples_per_second': 82.228, 'train_steps_per_second': 5.141, 'total_flos': 4863924917987688.0, 'train_loss': 0.17489797343378483, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, doc_id. If text, doc_id are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4507
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2340: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "


Trainer is attempting to log a value of "{'f1': 0.9040736700681102}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9110992459192536}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.9030397159973375}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'epoch': 5.0,
 'eval_f1': {'f1': 0.9040736700681102},
 'eval_loss': 0.37183845043182373,
 'eval_precision': {'precision': 0.9110992459192536},
 'eval_recall': {'recall': 0.9030397159973375},
 'eval_runtime': 15.7997,
 'eval_samples_per_second': 285.259,
 'eval_steps_per_second': 17.848}